## Paso 1: las especialidades de tercer nivel pasan a su padre

In [1]:
import os
import shutil

In [2]:
files_xml_specialties = 'specialties_subespecialties_xml'
path_output_specialties = 'specialties_joined_xml'

files_xml_case_report = 'specialties_subespecialties_case_report_xml'
path_output_cr = 'specialties_joined_case_report_xml'

files_xml_specialties_remove = 'remove_specialty'
files_xml_cr_remove = 'remove_specialty_case_report'

In [3]:
def obtain_name_father(name_specialty_child):
    name_specialty_child = name_specialty_child.replace(".xml", "")
    name_specialty_child = name_specialty_child.split("_")[0][:-4]
    
    for specialty_father in os.listdir(files_xml_specialties):
        if specialty_father.startswith(name_specialty_child+ '_'):
            return specialty_father
    
    return ""
    

In [5]:
def is_level_third(name_specialty):
    name_specialty = name_specialty.replace(".xml", "")
    array_name_specialty = name_specialty.split(".")
    
    if len(array_name_specialty) == 5:
        return True
    
    if len(array_name_specialty) == 6:
        print("CUIDADO 4 nivel", name_specialty)
        return False

    return False

In [6]:
def remove_specialty(specialty):

    shutil.move(os.path.join(files_xml_specialties,specialty), os.path.join(files_xml_specialties_remove,specialty))
    shutil.move(os.path.join(files_xml_case_report,specialty), os.path.join(files_xml_cr_remove,specialty))
    print("Eliminado:", specialty)
    

In [7]:
def read_specialties():
    for specialty in os.listdir(files_xml_specialties):
        third = is_level_third(specialty)
        
        # si es tercer nivel entonces pasamos los ficheros al segundo nivel
        if third:

            # cogemos el contenido del hijo NO CASO REPORT
            path_child = os.path.join(files_xml_specialties, specialty)
            fchild = open(path_child , 'r')
            child_content = fchild.read()
            fchild.close()
            
            
            # cogemos el contenido del hijo CASO REPORT
            path_child = os.path.join(files_xml_case_report, specialty)
            fchild = open(path_child , 'r')
            child_content_cr = fchild.read()
            fchild.close()
            
            
            # especialidad padre
            specialty_father = obtain_name_father(specialty)
            print("3er nivel:", specialty, "-> padre:", specialty_father)

            if len(specialty_father) > 1:
                
                # fichero join
                path_join = os.path.join(path_output_specialties, specialty_father)
                path_join_case_report = os.path.join(path_output_cr, specialty_father)
            
                # si el fichero join no existe
                if not os.path.isfile(path_join):
    
                    # contenido del padre NO CR
                    shutil.copy(os.path.join(files_xml_specialties,specialty_father), path_join)
                    fjoin = open(path_join , 'a')
                    fjoin.write('\n')
                    fjoin.write(child_content)
                    fjoin.close()
                    
                    
                    # contenido del padre CR
                    shutil.copy(os.path.join(files_xml_case_report,specialty_father), path_join_case_report)
                    fjoin = open(path_join_case_report , 'a')
                    fjoin.write('\n')
                    fjoin.write(child_content_cr)
                    fjoin.close()
                    
                    print("Añadido de 3 y 2 nivel juntos en: ", specialty_father)
                    remove_specialty(specialty)
                    
                # si el fichero ya existe, solo sumamos el hijo
                else:
                    
                    # añadimos el contenido del hijo SIN CR
                    fjoin = open(path_join , 'a')
                    fjoin.write('\n')
                    fjoin.write(child_content)
                    fjoin.close()
                    
                    # añadimos el contenido del hijo CR
                    fjoin = open(path_join_case_report , 'a')
                    fjoin.write('\n')
                    fjoin.write(child_content_cr)
                    fjoin.close()

                    print("Añadido de 3 y 2 nivel juntos en: ", specialty_father)
                    remove_specialty(specialty)


In [8]:
read_specialties()

3er nivel: H02.403.429.515.750_surgical_oncology.xml -> padre: H02.403.429.515_medical_oncology.xml
Añadido de 3 y 2 nivel juntos en:  H02.403.429.515_medical_oncology.xml
Eliminado: H02.403.429.515.750_surgical_oncology.xml
3er nivel: H02.403.429.515.250_psycho_oncology.xml -> padre: H02.403.429.515_medical_oncology.xml
Añadido de 3 y 2 nivel juntos en:  H02.403.429.515_medical_oncology.xml
Eliminado: H02.403.429.515.250_psycho_oncology.xml
3er nivel: H02.403.720.750.250_environmental_medicine.xml -> padre: H02.403.720.750_preventive_medicine.xml
Añadido de 3 y 2 nivel juntos en:  H02.403.720.750_preventive_medicine.xml
Eliminado: H02.403.720.750.250_environmental_medicine.xml
3er nivel: H02.403.680.600.937_telerehabilitation.xml -> padre: H02.403.680.600_rehabilitation.xml
Añadido de 3 y 2 nivel juntos en:  H02.403.680.600_rehabilitation.xml
Eliminado: H02.403.680.600.937_telerehabilitation.xml
3er nivel: H02.403.429.515.500_radiation_oncology.xml -> padre: H02.403.429.515_medical_on

### Paso 1.1. modificar los fichero para que sean xml legibles

In [22]:
def good_xml_specialties(path_specialties, path_output_cr):
    '''
    for specialty in os.listdir(path_specialties):
        
        # construímos el fichero correcto 
        ffail = open(os.path.join(path_specialties, specialty),"r")
        content_fail = ffail.read()
        ffail.close()

        if '</PubmedArticleSet>\n<?xml version="1.0" ?>\n<!DOCTYPE PubmedArticleSet PUBLIC "-//NLM//DTD PubMedArticle, 1st January 2019//EN" "https://dtd.nlm.nih.gov/ncbi/pubmed/out/pubmed_190101.dtd">\n<PubmedArticleSet>' in content_fail:
            print("Modificando especialidad", specialty)
            content_well = content_fail.replace('</PubmedArticleSet>\n<?xml version="1.0" ?>\n<!DOCTYPE PubmedArticleSet PUBLIC "-//NLM//DTD PubMedArticle, 1st January 2019//EN" "https://dtd.nlm.nih.gov/ncbi/pubmed/out/pubmed_190101.dtd">\n<PubmedArticleSet>', '')

            fout = open(os.path.join(path_output_specialties, specialty.replace(".xml", ".good")),"w")
            fout.write(content_well)
            fout.close()

        else:
            print(specialty, "No existe")
    '''
    for specialty in os.listdir(path_output_cr):
        
        # construímos el fichero correcto 
        ffail = open(os.path.join(path_output_cr, specialty),"r")
        content_fail = ffail.read()
        ffail.close()

        if '</PubmedArticleSet>\n<?xml version="1.0" ?>\n<!DOCTYPE PubmedArticleSet PUBLIC "-//NLM//DTD PubMedArticle, 1st January 2019//EN" "https://dtd.nlm.nih.gov/ncbi/pubmed/out/pubmed_190101.dtd">\n<PubmedArticleSet>' in content_fail:
            print("Modificando especialidad", specialty)
            content_well = content_fail.replace('</PubmedArticleSet>\n<?xml version="1.0" ?>\n<!DOCTYPE PubmedArticleSet PUBLIC "-//NLM//DTD PubMedArticle, 1st January 2019//EN" "https://dtd.nlm.nih.gov/ncbi/pubmed/out/pubmed_190101.dtd">\n<PubmedArticleSet>', '')
            del content_fail
            
            fout = open(os.path.join(path_output_cr, specialty.replace(".xml", ".good")),"w")
            fout.write(content_well)
            fout.close()
            
            del content_well

        else:
            print(specialty, "No existe")


In [23]:
path_specialties = 'specialties_joined_xml'
path_output_cr = 'specialties_joined_case_report_xml'
good_xml_specialties(path_specialties, path_output_cr)

Modificando especialidad H02.403.429_internal_medicine.xml
H02.403.830_sports_medicine.xml No existe
Modificando especialidad H02.403.600_neurology.xml
Modificando especialidad H02.403.810_specialties_surgical.xml
Modificando especialidad H02.403.650_pathology.xml
Modificando especialidad H02.403.690_psychiatry.xml
Modificando especialidad H02.403.763_reproductive_medicine.xml
Modificando especialidad H02.403.330_forensic_medicine.xml
Modificando especialidad H02.403.740_radiology.xml
H02.403.670_pediatrics.xml No existe
H02.403.560_naval_medicine.xml No existe
H02.403.250_emergency_medicine.xml No existe


# Paso 2. Si el segundo nivel tiene menos de 1000 doc entonces pasa al padre

Hasta aquí, hemos guardado los ficheros en zip llamados: step1... (por si hubiera algun problema)

In [11]:
import os
import shutil
import xml.etree.ElementTree as ET

In [12]:
# WITHIN CASES REPORTS
files_xml_specialties = 'specialties_subespecialties_xml'
path_output_specialties = 'specialties_joined_xml'
files_xml_specialties_remove = 'remove_specialty'


# CASES REPORTS
files_xml_case_report = 'specialties_subespecialties_case_report_xml'
path_output_cr = 'specialties_joined_case_report_xml'
files_xml_cr_remove = 'remove_specialty_case_report'

In [13]:
def remove_specialty(specialty):

    shutil.move(os.path.join(files_xml_specialties,specialty), os.path.join(files_xml_specialties_remove,specialty))
    shutil.move(os.path.join(files_xml_case_report,specialty), os.path.join(files_xml_cr_remove,specialty))
    print("Eliminado:", specialty)

In [14]:
def is_level_second(name_specialty):
    name_specialty = name_specialty.replace(".xml", "")
    array_name_specialty = name_specialty.split(".")
    
    if len(array_name_specialty) == 4:
        return True
    
    return False

In [15]:
def obtain_name_father(name_specialty_child):
    name_specialty_child = name_specialty_child.replace(".xml", "")
    name_specialty_child = name_specialty_child.split("_")[0][:-4]
    
    for specialty_father in os.listdir(files_xml_specialties):
        if specialty_father.startswith(name_specialty_child+ '_'):
            return specialty_father
    
    return ""

In [16]:
def get_lists_pmid_title_abstract(root):
    
    cont_pmid = 0
    cont_title = 0
    cont_abstract = 0
    
    for PubmedArticle in root.findall('PubmedArticle'):
        
        for MedlineCitation in PubmedArticle.findall('MedlineCitation'):
            pmid = MedlineCitation.find('PMID').text
            cont_pmid += 1
        
            for Article in MedlineCitation.findall('Article'):
                if not Article.find('VernacularTitle') is None:              
                    cont_title += 1
            
            for OtherAbstract in MedlineCitation.findall('OtherAbstract'):
                abstrac_lang = OtherAbstract.get('Language')
                                
                if abstrac_lang == 'spa':
                    cont_abstract += 1
        
    return cont_pmid, cont_title, cont_abstract

In [17]:
def read_xml_file(file_xml):
    tree = ET.parse(file_xml)
    root = tree.getroot()
    return root

In [18]:
def read_specialties():
    for specialty in os.listdir(files_xml_specialties):
        
        second = is_level_second(specialty)
        
        if second:
            # contar ficheros
            xml_data = read_xml_file(os.path.join(files_xml_specialties, specialty))
            cont_pmid, cont_title, cont_abstract = get_lists_pmid_title_abstract(xml_data)
            
            if cont_title < 1000:
                specialty_father = obtain_name_father(specialty)
                print("Segundo nivel:", specialty, "nº titulos:", cont_title, " padre:", specialty_father )
                
                if len(specialty_father) > 1:
                    
                    # cogemos el contenido del hijo sin CASE REPORT
                    path_child = os.path.join(files_xml_specialties, specialty)
                    fchild = open(path_child , 'r')
                    child_content = fchild.read()
                    fchild.close()
                    
                    # cogemos el contenido del hijo CASE REPORT
                    path_child_cr = os.path.join(files_xml_case_report, specialty)
                    fchild = open(path_child_cr , 'r')
                    child_content_cr = fchild.read()
                    fchild.close()
                
                    # fichero join de salida
                    path_join = os.path.join(path_output_specialties, specialty_father)
                    path_join_case_report = os.path.join(path_output_cr, specialty_father)

                    # si el fichero join no existe
                    if not os.path.isfile(path_join):
                            
                        # copiamos el padre y añadimos el hijo
                        shutil.copy(os.path.join(files_xml_specialties,specialty_father), path_join)
                        fjoin = open(path_join , 'a')
                        fjoin.write('\n')
                        fjoin.write(child_content)
                        fjoin.close()
                        
                         # contenido del padre CR
                        shutil.copy(os.path.join(files_xml_case_report,specialty_father), path_join_case_report)
                        fjoin = open(path_join_case_report , 'a')
                        fjoin.write('\n')
                        fjoin.write(child_content_cr)
                        fjoin.close()

                        remove_specialty(specialty)
                        
                        #print("Borrada la espe ", specialty_father)

                    # si el fichero ya existe, solo sumamos el hijo
                    else:
                        # añadimos el contenido del hijo SIN CR
                        fjoin = open(path_join , 'a')
                        fjoin.write('\n')
                        fjoin.write(child_content)
                        fjoin.close()

                        # añadimos el contenido del hijo CR
                        fjoin = open(path_join_case_report , 'a')
                        fjoin.write('\n')
                        fjoin.write(child_content_cr)
                        fjoin.close()

                        remove_specialty(specialty)

                        #print("Modificado el 2 nivel con menos de 1000 docs en: ", specialty_father)
                

In [19]:
read_specialties()

Segundo nivel: H02.403.763.500_andrology.xml nº titulos: 559  padre: H02.403.763_reproductive_medicine.xml
Eliminado: H02.403.763.500_andrology.xml
Segundo nivel: H02.403.250.500_pediatric_emergency_medicine.xml nº titulos: 7  padre: H02.403.250_emergency_medicine.xml
Eliminado: H02.403.250.500_pediatric_emergency_medicine.xml
Segundo nivel: H02.403.810.796_surgical_oncology.xml nº titulos: 23  padre: H02.403.810_specialties_surgical.xml
Eliminado: H02.403.810.796_surgical_oncology.xml
Segundo nivel: H02.403.650.510_pathology_surgical.xml nº titulos: 19  padre: H02.403.650_pathology.xml
Eliminado: H02.403.650.510_pathology_surgical.xml
Segundo nivel: H02.403.429.865_sleep_medicine_specialty.xml nº titulos: 8  padre: H02.403.429_internal_medicine.xml
Eliminado: H02.403.429.865_sleep_medicine_specialty.xml
Segundo nivel: H02.403.670.450_pediatric_emergency_medicine.xml nº titulos: 7  padre: H02.403.670_pediatrics.xml
Eliminado: H02.403.670.450_pediatric_emergency_medicine.xml
Segundo niv

### Paso 2.1. modificar los fichero para que sean xml legibles

## Paso 3. Comprobación

In [2]:
import os
import shutil

In [3]:
path_input = 'specialties_subespecialties_xml'

In [4]:
def is_level_first_second(name_specialty):
    name_specialty = name_specialty.replace(".xml", "")
    array_name_specialty = name_specialty.split(".")
    
    if len(array_name_specialty) == 4 or len(array_name_specialty) == 3:
        return True
    
    return False

In [5]:
def read_xml_specialties():
    for specialty in os.listdir(path_input):
        
        correct = is_level_first_second(specialty)
        if correct:
            
            print("Cooorrecta", specialty)
        else:
            print("**** error", specialty)
        

In [6]:
read_xml_specialties()

Cooorrecta H02.403.810.494_orthopedics.xml
Cooorrecta H02.403.350_genetics_medical.xml
Cooorrecta H02.403.645_palliative_medicine.xml
Cooorrecta H02.403.879_tropical_medicine.xml
Cooorrecta H02.403.429.675_pulmonary_medicine.xml
Cooorrecta H02.403.371_global_health.xml
Cooorrecta H02.403.429.480_infectious_disease_medicine.xml
Cooorrecta H02.403.720.750_preventive_medicine.xml
Cooorrecta H02.403.670.400_neonatology.xml
Cooorrecta H02.403.429_internal_medicine.xml
Cooorrecta H02.403.220_community_medicine.xml
Cooorrecta H02.403.230_disaster_medicine.xml
Cooorrecta H02.403.690.150_community_psychiatry.xml
Cooorrecta H02.403.830_sports_medicine.xml
Cooorrecta H02.403.810.450_obstetrics.xml
Cooorrecta H02.403.500_military_medicine.xml
Cooorrecta H02.403.894_vaccinology.xml
Cooorrecta H02.403.600_neurology.xml
Cooorrecta H02.403.959_wilderness_medicine.xml
Cooorrecta H02.403.800_social_medicine.xml
Cooorrecta H02.403.720_public_health.xml
Cooorrecta H02.403.429.405_gastroenterology.xml
Cooo